## Print @count statistics from network
Helmet 4.0 testing:
* Get network attributes
* Print statistics and barplots

Author: Atte Supponen, WSP, 2020

In [9]:
from IPython.display import display, HTML, Image
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Connect Emme-project, emmebank, scenario and network
_m = inro.modeller
modeller = _m.Modeller()
desktop = modeller.desktop
emmebank = modeller.emmebank
data_explorer = desktop.data_explorer()

# Emme tools
create_attr = modeller.tool("inro.emme.data.extra_attribute.create_extra_attribute")
import_attr = modeller.tool("inro.emme.data.extra_attribute.import_extra_attributes")


# Palettes fro plotting
palette1 = ['#97C798', '#B8F4BA', '#B9E5F4', '#F7B7B8']
palette2 = ['#42a600', '#56d900', '#0042A6', '#a60042']


## Read input obs data

In [33]:
scenarios = [21, 22, 23]
scenarios_filenames = ["links_2016_aht.txt", "links_2016_pt.txt", "links_2016_iht.txt"]

for i, j in zip(scenarios, scenarios_filenames):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@l_count",
                extra_attribute_description="observed counts",
                overwrite=True)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@m_count",
                extra_attribute_description="observed counts",
                overwrite=True)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@l_count_type",
                extra_attribute_description="observed counts types",
                overwrite=True)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@direction",
                extra_attribute_description="observed counts direction",
                overwrite=True)

    default_path = os.path.dirname(_m.Modeller().emmebank.path).replace("\\","/")

    file_path = os.path.join(default_path, "../Scripts/notebook/", j).replace("\\","/")

    import_attr(file_path,
                scenario = scenario_selection,
                field_separator=",",
                column_labels={0: "inode", 1: "jnode", 2: "@l_count", 3: "@l_count_type", 4: "@m_count", 5: "@direction"},
                revert_on_error=True)

### Get network data to pandas dataframe


In [4]:
# parameters
scenarios_ht = [21, 23]
scenarios_names = [u"2016_aht", u"2016_iht"]

scenarios = []
ids = []
l_counts = []
l_counts_types = []
directions = []

car_leisure = []
car_work = []
truck = []
trailer_truck = []
van = []
bus = []


for i, j in zip(scenarios_ht, scenarios_names):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    scenario = modeller.scenario
    network = scenario.get_network()
    #
    # list @code extra attribuuttiin tallennettuista vaylaosan tunnuksista
    for link in network.links():
        if link['@l_count'] > 0:
            scenarios.append(j)
            ids.append(link.id)
            l_counts.append(link['@l_count'])
            l_counts_types.append(link['@l_count_type'])
            directions.append(link['@direction'])
            
            car_leisure.append(link['@car_leisure'])
            car_work.append(link['@car_work'])
            truck.append(link['@truck'])
            trailer_truck.append(link['@trailer_truck'])
            van.append(link['@van'])
            bus.append(link['@bus'])

    l_sum = (np.array(car_leisure) + 
             np.array(car_work) + 
             np.array(truck) + 
             np.array(trailer_truck) + 
             np.array(van),
             np.array(bus)
            )
    
d = {'scenario': scenarios,
     'id': ids, 
     'l_count': l_counts, 
     'l_count_type': l_counts_types, 
     'volau_ul3': l_sum, 
     'direction': directions} 
            
df = pd.DataFrame(data = d) 
df.to_csv("l_count_model.txt", index = False, sep='\t')

ValueError: arrays must all be same length

In [ ]:
df

In [42]:
palette1 = ['#97C798', '#B8F4BA', '#B9E5F4', '#F7B7B8']
palette2 = ['#42a600', '#56d900', '#0042A6', '#a60042']

In [10]:
directions_names = {"direction": [1,2,3,4],
                   "direction_names": ["keskustaan","muualle","lanteen","itaan"]
                   }
directions_names = pd.DataFrame(data = directions_names)
directions_names = directions_names.set_index('direction')
directions_names.loc[direction].values

NameError: name 'direction' is not defined

In [11]:
l_count_names = {"l_count_type":[1,2,3,4,5,6,7],
                "l_count_name":["Niemen raja","Kantakaupungin raja","Kaupungin raja","Poikittaislinja","Keha I","Keha II","Keha III"]}
l_count_names = pd.DataFrame(data = l_count_names)
l_count_names = l_count_names.set_index('l_count_type')
l_count_names.loc[dfPlot.loc[scenario,direction].index.values].filter(['l_count_name'])

NameError: name 'dfPlot' is not defined

In [12]:
dfPlot = df.groupby(['scenario','direction','l_count_type'])['l_count','volau_ul3'].sum()
dfPlot


l_count     volau_ul3
scenario direction l_count_type                       
2016_aht 1.0       1.0           10496.0   8140.368581
                   2.0           18763.0  16412.075020
                   3.0           34725.0  35502.715088
         2.0       1.0            6133.0   4928.800865
                   2.0            9536.0   7170.553939
                   3.0           23290.0  20875.629895
         3.0       4.0            8207.0  10957.572678
                   5.0            8970.0  13003.033203
                   6.0            4964.0   3700.277466
                   7.0           14095.0  11811.437480
         4.0       4.0            9800.0  10835.325472
                   5.0           14190.0  15848.932861
                   6.0            5365.0   4033.439819
                   7.0           13117.0  11963.884644
2016_iht 1.0       1.0            7157.0   6406.586521
                   2.0           11208.0  10190.333065
                   3.0           27568.0  24541.809146
         2.0       1.0           10085.0   8028.200611
                   2.0           18663.0  15489.903454
                   3.0           36002.0  35577.164305
         3.0       4.0            9652.0  11420.232787
                   5.0            9838.0  12237.846599
                   6.0            4953.0   4244.517736
                   7.0           14115.0  12247.340902
         4.0       4.0            9687.0  11258.376244
                   5.0           12839.0  17862.927002
                   6.0            4671.0   3272.875768
                   7.0           15793.0  13319.114054

In [13]:
image_export_path = os.path.join("./Output/")
if not os.path.exists(image_export_path):
    os.mkdir(image_export_path)

for scenario in list(dict.fromkeys(scenarios)):
    for direction in list(dict.fromkeys(directions)):
        
        fig, ax = plt.subplots(figsize = [9,5], dpi = 300)

        # set width of bar
        barWidth = 0.3
 
        # set height of bar
        bars_count = dfPlot.loc[scenario,direction].filter(['l_count']).values
        bars_volau = dfPlot.loc[scenario,direction].filter(['volau_ul3']).values
        

        # Set position of bar on X axis
        r1 = np.arange(len(bars_count))
        r2 = [x + barWidth for x in r1]

        # Make the plot
        ax_bars_count = ax.bar(r1, bars_count, color=palette1[0], width=barWidth, edgecolor='white', label=u'Laskenta', alpha = 0.9)
        ax_bars_volau = ax.bar(r2, bars_volau, color=palette1[1], width=barWidth, edgecolor='white', label=u'Mallinnus', alpha = 0.9)
        
        # Add xticks on the middle of the group bars
        plt.xticks([r + barWidth*0.5 for r in range(len(bars_count))], l_count_names.loc[dfPlot.loc[scenario,direction].index.values].filter(['l_count_name']).values)
        plt.tick_params(axis='both', which='major', labelsize=11)
        
        # axis names
        plt.xlim(-0.3, len(bars_count))
        plt.ylim(0, 50000) 
        plt.ylabel(u'Moottoriajoneuvoa', fontsize = 11)
        
        title = str(scenario).replace('_',' ') + ' ' + str(directions_names.loc[direction].values).strip("['']")
        plt.title(u'Liikennemäärät %s' % (title),fontweight="bold")
        
        # spines invisible
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        plt.tick_params(top='off', right='off', labelleft='on', labelbottom='on')
        
        # Shrink current axis's height by 10% on the bottom
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.15,
                         box.width, box.height * 0.85])
        
        # Put a legend below current axis
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon= False, fontsize = 11)
        
        def autolabel(bars):
            """
            Attach a text label above each bar displaying its height
            """
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., 1+height,
                        '%d' % int(round(height, -2)),
                        ha='center', va='bottom', color="black", fontsize=11)

        autolabel(ax_bars_count)
        autolabel(ax_bars_volau)
        
        filename = str(scenario) + '_' + str(direction)
        
        plt.savefig('./Output/%s_trips.png' % (filename.strip()))
        plt.close()
        

In [ ]:
scenarios = []
ids = []
modes = []
m_counts = []
m_counts_types = []
voltr = []
directions = []

for i, j in zip(scenario_nos, scenario_names):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    scenario = modeller.scenario
    network = scenario.get_network()
    #
    # list @code extra attribuuttiin tallennettuista vaylaosan tunnuksista
    for link in network.links():
        if link['@m_count'] > 0:
            voltr_segment = 0
            for segment in link.segments():
                voltr_segment += segment.transit_volume
            scenarios.append(j)
            ids.append(link.id)
            modes.append(link.modes)
            m_counts.append(link['@m_count'])
            m_counts_types.append(link['@l_count_type'])
            voltr.append(voltr_segment)
            directions.append(link['@direction'])
        
    
    volau_ul3 = np.array(volau) + np.array(ul3)
    
    
    
    
d = {'scenario': scenarios,
     'id': ids, 
     'm_count': m_counts, 
     'm_count_type': m_counts_types, 
     'mode': [list(x) for x in modes],
     'voltr': voltr, 
     'direction': directions} 
            
df = pd.DataFrame(data = d) 
df.to_csv("m_count_model.txt", index = False, sep='\t')

In [ ]:
fig, ax = plt.subplots(figsize = [9,5], dpi = 300)

# set width of bar
barWidth = 0.2
 
# set height of bar
bars_jk = gs1_matkat_sums.filter(regex='JK')
bars_pp = gs1_matkat_sums.filter(regex='PP')
bars_jl = gs1_matkat_sums.filter(regex='JL')
bars_ha = gs1_matkat_sums.filter(regex='HA')
 
# Set position of bar on X axis
r1 = np.arange(len(bars_jk))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
 
# Make the plot
ax_bars_jk = ax.bar(r1, bars_jk, color=palette1[0], width=barWidth, edgecolor='white', label=u'Jalankulku', alpha = 0.9)
ax_bars_pp = ax.bar(r2, bars_pp, color=palette1[1], width=barWidth, edgecolor='white', label=u'Polkupyörä', alpha = 0.9)
ax_bars_jl = ax.bar(r3, bars_jl, color=palette1[2], width=barWidth, edgecolor='white', label=u'Joukkoliikenne', alpha = 0.9)
ax_bars_ha = ax.bar(r4, bars_ha, color=palette1[3], width=barWidth, edgecolor='white', label=u'Henkilöauto', alpha = 0.9)

# Add xticks on the middle of the group bars
plt.xticks([r + barWidth*2 for r in range(len(bars_jk))], scenario_names)
plt.tick_params(axis='both', which='major', labelsize=11)

# axis names
plt.xlim(-0.3, len(scenario_names))
plt.ylim(0, 200000) 
plt.ylabel(u'Lähtevää matkaa / vrk', fontsize = 11)

plt.title(u'Helsingin niemeltä lähtevien matkojen määrä (vrk)',fontweight="bold")

# spines invisible
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tick_params(top='off', right='off', labelleft='on', labelbottom='on')

# Shrink current axis's height by 10% on the bottom
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.15,
                 box.width, box.height * 0.85])

# Put a legend below current axis
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon= False, fontsize = 11)

def autolabel(bars):
    """
    Attach a text label above each bar displaying its height
    """
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., 1+height,
                '%d' % int(round(height, -2)),
                ha='center', va='bottom', color="black", fontsize=11)

autolabel(ax_bars_ha)

image_export_path = os.path.join("./Output/")
if not os.path.exists(image_export_path):
    os.mkdir(image_export_path)

plt.savefig('./Output/gs1_%s_trips.png' % (filename))
plt.close()